In [1]:
import requests
import pandas as pd
from config import census_key
import json

In [2]:
# Read in zip codes
zips = pd.read_csv("../Zip_Codes_Chicago.csv")
zips

,the_geom,OBJECTID,ZIP,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.67762151065281 41.91775780...,33,60647,1.060523e+08,42720.044406
1,MULTIPOLYGON (((-87.72683253163021 41.92264626...,34,60639,1.274761e+08,48103.782721
2,MULTIPOLYGON (((-87.78500237831095 41.90914785...,35,60707,4.506904e+07,27288.609612
3,MULTIPOLYGON (((-87.6670686895295 41.888851884...,36,60622,7.085383e+07,42527.989679
4,MULTIPOLYGON (((-87.70655631674127 41.89555340...,37,60651,9.903962e+07,47970.140153
...,...,...,...,...,...
56,MULTIPOLYGON (((-87.69479284658739 41.83007507...,57,60623,1.552855e+08,53406.915617
57,MULTIPOLYGON (((-87.6830596746508 41.757859474...,58,60629,2.111148e+08,58701.325375
58,MULTIPOLYGON (((-87.62372919454222 41.72167067...,59,60620,2.116961e+08,58466.160298
59,MULTIPOLYGON (((-87.57691454500865 41.79510541...,60,60637,1.254243e+08,52377.854541


In [3]:
# Remove unneeded columns. This will create an error if rows have already been dropped
zips_trim = zips[['OBJECTID','ZIP']]

In [4]:
# Just checking in 
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(zips_trim)

    OBJECTID    ZIP
0         33  60647
1         34  60639
2         35  60707
3         36  60622
4         37  60651
5         38  60611
6         39  60638
7         40  60652
8          1  60626
9          8  60615
10         9  60621
11         2  60645
12        13  60643
13         3  60660
14         4  60640
15        32  60614
16        41  60631
17        42  60646
18        43  60628
19         5  60625
20         6  60641
21         7  60657
22        10  60636
23        11  60649
24        12  60617
25        14  60633
26        15  60643
27        16  60612
28        48  60604
29        17  60624
30        47  60656
31        18  60644
32        49  60655
33        19  60603
34        20  60605
35        21  60653
36        22  60609
37        23  60666
38        24  60618
39        25  60616
40        26  60602
41        27  60601
42        28  60608
43        29  60607
44        30  60661
45        31  60606
46        44  60827
47        45  60630
48        46  60642


In [5]:
# reset print options
pd.reset_option('all')
zips_trim


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



C:\Users\Ward33\anaconda3\lib\site-packages\pandas\_config\config.py:620: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


,OBJECTID,ZIP
0,33,60647
1,34,60639
2,35,60707
3,36,60622
4,37,60651
...,...,...
56,57,60623
57,58,60629
58,59,60620
59,60,60637


In [ ]:
# API call loop for table B14005 2018
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs5?'
group = "B15003"
zip_ = ""
selectB15003_full = []
# loop through each zip code
for index, row in zips_trim.iterrows():
    zip_ = row['ZIP']
    try:
        print(f"Trying {row['ZIP']}")
        response = requests.get(f"{base_url}get=group({group})&for=zip%20code%20tabulation%20area:{zip_}&key={census_key}").json()
    except:
        print(f"failed with this address: {base_url}get=group({group})&for=zip%20code%20tabulation%20area:{zip_}&key={census_key}")
    # Clean call response
    response_df = pd.DataFrame(response)
    response_df.columns = response_df.iloc[0]
    # Select needed columns
    selectB15003 = pd.DataFrame(response_df.iloc[1][['GEO_ID',
                    'B15003_001E','B15003_002E','B15003_003E','B15003_004E','B15003_005E','B15003_006E',
                    'B15003_007E','B15003_008E','B15003_009E','B15003_010E',
                    'B15003_011E','B15003_012E','B15003_013E',
                     'B15003_014E','B15003_015E','B15003_016E','B15003_017E',
                    'B15003_018E','B15003_019E', 'B15003_020E','B15003_021E',
                    'B15003_022E','B15003_023E','B15003_024E','B15003_025E']])
    select2B15003 = selectB15003.transpose()
    # store selected response
    selectB15003_full.append(select2B15003.iloc[0])
# put all responses into a dataframe
selectB15003_full_df = pd.DataFrame(selectB15003_full)

selectB15003_full_df 
    

Trying 60647
Trying 60639
Trying 60707
Trying 60622
Trying 60651
Trying 60611
Trying 60638
Trying 60652
Trying 60626
Trying 60615


In [ ]:
# Create zip code column
selectB15003_full_df['ZIP'] = selectB15003_full_df['GEO_ID'].str[-5:]
selectB15003_full_df

In [ ]:
# Consolidate and rename columns
selectB15003_full_df.rename(columns = {'B15003_001E':'Total_Pop', 'B15003_002E':'No_School','B15003_012E':'8th_Grade',
                                      'B15003_021E':'Assoc_Deg','B15003_022E':'Bach_Deg','B15003_023E':'Mast_Deg',
                                       'B15003_024E':'Prof_Deg','B15003_025E':'Doct_Deg'}, inplace = True)
# Calculate total with some schooling below 8th grade
selectB15003_full_df['<8th_Grade'] = selectB15003_full_df['B15003_003E'].astype('int64') + selectB15003_full_df['B15003_004E'].astype('int64')
+ selectB15003_full_df['B15003_005E'].astype('int64') + selectB15003_full_df['B15003_006E'].astype('int64')
+ selectB15003_full_df['B15003_007E'].astype('int64') + selectB15003_full_df['B15003_008E'].astype('int64')
+ selectB15003_full_df['B15003_009E'].astype('int64') + selectB15003_full_df['B15003_010E'].astype('int64')
+ selectB15003_full_df['B15003_011E'].astype('int64')
# Calculate total with some high school (9-12 grades inclusive) without diploma
selectB15003_full_df['Some_HS'] = selectB15003_full_df['B15003_013E'].astype('int64') + selectB15003_full_df['B15003_014E'].astype('int64')
+ selectB15003_full_df['B15003_015E'].astype('int64') + selectB15003_full_df['B15003_016E'].astype('int64')
# Calculate total with diploma, GED, or equiv.
selectB15003_full_df['HS_or_GED'] = selectB15003_full_df['B15003_017E'].astype('int64') + selectB15003_full_df['B15003_018E'].astype('int64')
# Calcucate total with some college without degree
selectB15003_full_df['Some_College'] = selectB15003_full_df['B15003_019E'].astype('int64') + selectB15003_full_df['B15003_020E'].astype('int64')

selectB15003_full_df

In [ ]:
# Derive percentages
selectB15003_full_df['No_School_Pct'] = (selectB15003_full_df['No_School'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['<8th_Grade_Pct'] = (selectB15003_full_df['<8th_Grade'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['8th_Grade_Pct'] = (selectB15003_full_df['8th_Grade'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Some_HS_Pct'] = (selectB15003_full_df['Some_HS'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['HS_GED_Pct'] = (selectB15003_full_df['HS_or_GED'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Some_Coll_Pct'] = (selectB15003_full_df['Some_College'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Assoc_Deg_Pct'] = (selectB15003_full_df['Assoc_Deg'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Bach_Deg_Pct'] = (selectB15003_full_df['Bach_Deg'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Mast_Deg_Pct'] = (selectB15003_full_df['Mast_Deg'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Prof_Deg_Pct'] = (selectB15003_full_df['Prof_Deg'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df['Doct_Deg_Pct'] = (selectB15003_full_df['Doct_Deg'].astype('float64') / selectB15003_full_df['Total_Pop'].astype('float64'))*100
selectB15003_full_df

In [ ]:
#reindex
selectB15003_full_df = selectB15003_full_df.reset_index(drop=True)

# Trim and reorder columns
selectB15003_trim_df = selectB15003_full_df[['ZIP','Total_Pop','No_School','No_School_Pct','<8th_Grade','<8th_Grade_Pct',
                                             '8th_Grade','8th_Grade_Pct',
                                            'Some_HS','Some_HS_Pct','HS_or_GED','HS_GED_Pct','Some_College','Some_Coll_Pct',
                                            'Assoc_Deg','Assoc_Deg_Pct','Bach_Deg','Bach_Deg_Pct','Mast_Deg','Mast_Deg_Pct',
                                            'Prof_Deg','Prof_Deg_Pct','Doct_Deg','Doct_Deg_Pct']]


selectB15003_trim_df

In [ ]:
# Export
selectB15003_trim_df.to_csv('Output/EduData.csv')